In [1]:
!pip install fastai
from fastai import *
from fastai.vision import *
# from fastai.callbacks.hooks import *
from fastai.utils.mem import *
from PIL import Image
from torchvision import transforms

In [2]:
import os
from pathlib import Path
import PIL.Image as PilImage

In [ ]:
#for Google Colab only
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !unzip data.zip
##rename unzipped file as data

Archive:  data.zip
   creating: project_Agri/bin_masks/
   creating: project_Agri/masks/
  inflating: project_Agri/labeled images/Copy of _04_05.bmp  
  inflating: project_Agri/labeled images/Copy of _04_15.bmp  
  inflating: project_Agri/labeled images/Copy of _04_08.bmp  
  inflating: project_Agri/images_green/Copy of _18_18.bmp  
  inflating: project_Agri/images_brown/Copy of _02_03.bmp  
  inflating: project_Agri/labeled images/Copy of _02_11.bmp  
  inflating: project_Agri/images_green/Copy of _18_01.bmp  
  inflating: project_Agri/labeled images/Copy of _03_04.bmp  
  inflating: project_Agri/labeled images/Copy of _01_10.bmp  
  inflating: project_Agri/labeled images/Copy of _02_03.bmp  
  inflating: project_Agri/labeled images/Copy of _18_01.bmp  
  inflating: project_Agri/labeled images/Copy of _02_14.bmp  
  inflating: project_Agri/labeled images/Copy of _13_19.bmp  
  inflating: project_Agri/images_brown/Copy of _02_14.bmp  
  inflating: project_Agri/labeled images/Copy of _0

### Uncomment as per platform used

In [4]:
#for Colab
# path=Path('/content/drive/My Drive/AI Stuff/project_Agri/') ## change <AI Stuff/... as per the location on drive>

#for GCP
path=Path('data') ## change 'data' as per relative location 

In [5]:
path_lbl = path/'labeled images'
path_img = path/'images_ndvi' # change as per image type: ndvi, green, brown
path_bm=path/'bin_masks'
path_masks=path/'masks'

### convert labeled images to binary images(B&W, 0/1 labels)

In [6]:
list1=get_image_files(path_lbl)
for path1 in list1:
  a=Image.open(path1)
  img=transforms.ToTensor()(a)
  a=img[0].flatten()#red channel
  b=[i>0.925 and i<0.93 for i in a] # red pixels are 0.9255 in red channel
  c=tensor(b).float().reshape(522,522).unsqueeze(0)
  # mask=PILMask.create(c)
  mask=transforms.ToPILImage()(c)
  stri=str(path1).split('/')[-1]
  mask.save(fp=path_bm/stri)

In [7]:
def getClassValues(label_names):

    containedValues = set([])

    for i in range(len(label_names)):
        tmp = open_mask(label_names[i])
        tmp = tmp.data.numpy().flatten()
        tmp = set(tmp)
        containedValues = containedValues.union(tmp)
        # print(type(containedValues))
    return list(containedValues)

def replaceMaskValuesFromZeroToN(mask, 
                                 containedValues):

    numberOfClasses = len(containedValues)
    newMask = np.zeros(mask.shape)

    for i in range(numberOfClasses):
        newMask[mask == containedValues[i]] = i
    
    return newMask

def convertMaskToPilAndSave(mask, 
                            saveTo,format='bmp'):

    imageSize = mask.squeeze().shape

    im = PilImage.new('L',(imageSize[1],imageSize[0]))
    im.putdata(mask.astype('uint8').ravel())
    im.save(saveTo,format=format)

def convertMasksToGrayscaleZeroToN(pathToLabels,saveToPath):
    label_names = get_image_files(pathToLabels)
    containedValues = getClassValues(label_names)

    for currentFile in label_names:
        currentMask = open_mask(currentFile).data.numpy()
        convertedMask = replaceMaskValuesFromZeroToN(currentMask, containedValues)
        convertMaskToPilAndSave(convertedMask, saveToPath/f'{currentFile.name}')
    
    print('Conversion finished!')

### Convert B&W images (pixel values 0 and 255) to Grayscale images (values 0,1, 2 and so on)

In [8]:
convertMasksToGrayscaleZeroToN(path_bm, path_masks)

Conversion finished!
